In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv.
2. Проведите предобработку:\
• Приведите тексты к нижнему регистру.\
• Замените все, кроме букв и цифр, на пробелы — это облегчит
дальнейшее разделение текста на слова. Для такой замены в
строке text подходит следующий вызов:\
re.sub('[^ a−zA−Z0−9]', '␣', text.lower())\
• Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).\
• Замените пропуски в столбцах LocationNormalized и ContractTime
на специальную строку ’nan’. Код для этого был приведен выше.\
• Примените DictVectorizer для получения one-hot-кодирования
признаков LocationNormalized и ContractTime.\
• Объедините все полученные признаки в одну матрицу "объектыпризнаки". Обратите внимание, что матрицы для текстов и
категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.
3. Обучите гребневую регрессию с параметром alpha=1. Целевая переменная записана в столбце SalaryNormalized.
4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv.
Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [2]:
!ls /content/gdrive/'MyDrive'/ML/

classification.csv  housing.names	  salary-train.csv  wine.data
comments.csv	    perceptron-test.csv   scores.csv	    wine.names
data-logistic.csv   perceptron-train.csv  svm-data.csv
housing.data	    salary-test-mini.csv  titanic.csv


In [4]:
import pandas as pd
import re
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack

In [22]:
data = pd.read_csv('/content/gdrive/MyDrive/ML/salary-train.csv')

In [24]:
data.head(5)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [25]:
def foo(X: pd.DataFrame) -> pd.DataFrame:
    X['FullDescription'] = pd.Series([x.lower() for x in X['FullDescription']])
    X['FullDescription'] = pd.Series([re.sub(r'[^a-zA-Z0-9]', ' ', x) for x in X['FullDescription']])
    X[['LocationNormalized', 'ContractTime']] = X[['LocationNormalized', 'ContractTime']].fillna('nan')

    return X

In [27]:
data = foo(data)
data.head(5)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


In [28]:
vectorizer = TfidfVectorizer(min_df=5)
features = vectorizer.fit_transform(data.FullDescription)
DVect = DictVectorizer()
LocNorm = data[['LocationNormalized', 'ContractTime']].to_dict('records')
vectLoc = DVect.fit_transform(LocNorm)

In [33]:
features

<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

In [30]:
vectLoc

<60000x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>

In [34]:
model = Ridge(alpha=1)
Xvectorized = hstack([features, vectLoc])
model.fit(X=Xvectorized, y=data.SalaryNormalized)

Ridge(alpha=1)

In [35]:
data_test = pd.read_csv('/content/gdrive/MyDrive/ML/salary-test-mini.csv')

In [36]:
data_test = foo(data_test)
features = vectorizer.transform(data_test.FullDescription)
LocNorm = data_test[['LocationNormalized', 'ContractTime']].to_dict('records')
vectLoc = DVect.transform(LocNorm)
Xvectorized = hstack([features, vectLoc])

In [37]:
pred = model.predict(Xvectorized)

предполагаемая зарплата

In [46]:
print(pred)

[56575.58558632 37135.92073248]
